In [2]:
import numpy as np
import os
import random
from copy import deepcopy
import matplotlib.pyplot as plt

class UltrametricTree:
    def __init__(self, n):
        self.n = n
        self.adj_matrix = np.zeros((2 * n - 1, 2 * n - 1))
        self.distances = np.zeros((2 * n - 1, 2 * n - 1))
        self.node_count = n

    def add_edge(self, i, j, weight):
        self.adj_matrix[i][j] = weight
        self.adj_matrix[j][i] = weight

    def update_distances(self):
        self.distances = np.copy(self.adj_matrix)
        for k in range(self.node_count):
            for i in range(self.node_count):
                for j in range(self.node_count):
                    if self.distances[i][j] > self.distances[i][k] + self.distances[k][j]:
                        self.distances[i][j] = self.distances[i][k] + self.distances[k][j]

    def compute_weight(self):
        return np.sum(self.adj_matrix) / 2

    def is_ultrametric(self):
        for i in range(self.node_count):
            for j in range(self.node_count):
                for k in range(self.node_count):
                    if self.distances[i][j] > max(self.distances[i][k], self.distances[k][j]):
                        return False
        root = 2 * self.n - 2
        leaf_distances = []
        for i in range(self.n):
            leaf_distances.append(self.distances[root][i])
        if len(set(leaf_distances)) != 1:
            return False
        return True

def generate_initial_tree(n):
    tree = UltrametricTree(n)
    for i in range(n - 1):
        tree.add_edge(i, i + 1, random.uniform(0.5, 1.5))
    tree.update_distances()
    return tree

def calculate_cost(tree):
    if not tree.is_ultrametric():
        return float('inf')
    return tree.compute_weight()

def generate_neighbor(tree):
    new_tree = deepcopy(tree)
    i, j = random.sample(range(new_tree.node_count), 2)
    new_weight = random.uniform(0.5, 1.5)
    new_tree.add_edge(i, j, new_weight)
    new_tree.update_distances()
    return new_tree

def convert_to_ultrametric_matrix(distance_matrix):
    n = len(distance_matrix)
    ultrametric_matrix = np.copy(distance_matrix)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                ultrametric_matrix[i][j] = min(ultrametric_matrix[i][j], max(ultrametric_matrix[i][k], ultrametric_matrix[k][j]))
    return ultrametric_matrix

def simulated_annealing(distance_matrix, initial_temp=10000, cooling_rate=0.99, stopping_temp=1, num_iters=1000):
    n = len(distance_matrix)
    initial_tree = generate_initial_tree(n)
    current_tree = initial_tree
    current_cost = calculate_cost(current_tree)
    best_tree = current_tree
    best_cost = current_cost
    
    temp = initial_temp
    values = [None for _ in range(num_iters)]
    
    for i in range(num_iters):
        neighbor_tree = generate_neighbor(current_tree)
        neighbor_cost = calculate_cost(neighbor_tree)
        
        if neighbor_cost < current_cost or random.uniform(0, 1) < np.exp((current_cost - neighbor_cost) / temp):
            current_tree = neighbor_tree
            current_cost = neighbor_cost
        
        if current_cost < best_cost:
            best_tree = current_tree
            best_cost = current_cost
        
        values[i] = best_cost
        temp *= cooling_rate
        
        if temp < stopping_temp:
            break
    
    return best_tree

def load_distance_matrix(file_path):
    return np.loadtxt(file_path, delimiter=' ')

def load_all_distance_matrices(directory_path):
    matrices = {}
    for file_name in os.listdir(directory_path):
        if file_name.startswith('matrix') and file_name.endswith('.txt'):
            file_path = os.path.join(directory_path, file_name)
            matrices[file_name] = load_distance_matrix(file_path)
    return matrices
    
directory_path = 'tests/' 
distance_matrices = load_all_distance_matrices(directory_path)

for file_name, distance_matrix in distance_matrices.items():
    print(f"Processing matrix from file: {file_name}")
    ultrametric_matrix = convert_to_ultrametric_matrix(distance_matrix)
    optimal_tree = simulated_annealing(ultrametric_matrix)
    if optimal_tree.is_ultrametric():
        print(optimal_tree.adj_matrix)
        print("Weight of the optimal tree:", optimal_tree.compute_weight())
    else:
        print("No ultrametric tree found.")


Processing matrix from file: matrix3.txt
[[0.         0.50009118 0.         0.77684638 0.         0.
  0.        ]
 [0.50009118 0.         0.55726984 0.         0.         0.
  0.        ]
 [0.         0.55726984 0.         0.5955952  0.         0.
  0.        ]
 [0.77684638 0.         0.5955952  0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.        ]]
Weight of the optimal tree: 2.4298025993929877
Processing matrix from file: matrix8.txt
[[0.         0.51823681 0.95074788 0.63784317 0.         0.
  0.         0.         0.        ]
 [0.51823681 0.         0.54392331 0.67105988 0.         0.
  0.         0.         0.        ]
 [0.95074788 0.54392331 0.         0.61473502 0.         0.
  0.         0.         0.        ]
 [0.63784317 0.67105988 0.61473502 0.         0.74822644 0.
  0.